# The Unreasonable Effectiveness of Character-level Language Models
# (and why RNNs are still cool)

## By [Yoav Goldberg](http://www.cs.biu.ac.il/~yogo) (2015)

#### (with minor changes by Peter Norvig (2022) for modern Python 3)

<hr>

[RNNs](https://en.wikipedia.org/wiki/Recurrent_neural_network), [LSTMs](https://en.wikipedia.org/wiki/Long_short-term_memory) and [Deep Learning](https://en.wikipedia.org/wiki/Deep_learning) are all the rage, and a recent [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by Andrej Karpathy is doing a great job explaining what these models are and how to train them.
It also provides some very impressive results of what they are capable of.  This is a great post, and if you are interested in natural language, machine learning or neural networks you should definitely read it. 

Go [**read it now**](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), then come back here. 

You're back? Good. Impressive stuff, huh? How could the network learn to imitate the input like that?
Indeed. I was quite impressed as well.

However, it feels to me that most readers of the post are impressed by the wrong reasons.
This is because they are not familiar with **unsmoothed maximum-liklihood character level language models** and their unreasonable effectiveness at generating rather convincing natural language outputs.

In what follows I will briefly describe these character-level maximum-likelihood langauge models, which are much less magical than RNNs and LSTMs, and show that they too can produce a rather convincing Shakespearean prose. I will also show about 30 lines of python code that take care of both training the model and generating the output. Compared to this baseline, the RNNs may seem somehwat less impressive. So why was I impressed? I will explain this too, below.

## Unsmoothed Maximum Likelihood Character Level Language Model 

The name is quite long, but the idea is very simple.  We want a model whose job is to guess the next character based on the previous *n* letters. For example, having seen `ello`, the next characer is likely to be either a commma or space (if we assume is is the end of the word "hello"), or the letter `w` if we believe we are in the middle of the word "mellow". Humans are quite good at this, but of course seeing a larger history makes things easier (if we were to see 5 letters instead of 4, the choice between space and `w` would have been much easier).

We will call *n*, the number of letters we need to guess based on, the _order_ of the language model.

RNNs and LSTMs can potentially learn infinite-order language model (they guess the next character based on a "state" which supposedly encode all the previous history). We here will restrict ourselves to a fixed-order language model.

So, we are seeing *n* letters, and need to guess the *n+1*th one. We are also given a large-ish amount of text (say, all of Shakespear works) that we can use. How would we go about solving this task?

Mathematically, we would like to learn a function *P(c* | *h)*. Here, *c* is a character, *h* is a *n*-letters history, and *P(c* | *h)* stands for how likely is it to see *c* after we've seen *h*.

Perhaps the simplest approach would be to just count and divide (a.k.a **maximum likelihood estimates**). We will count the number of times each letter *c* appeared after *h*, and divide by the total numbers of letters appearing after *h*. The **unsmoothed** part means that if we did not see a given letter following *h*, we will just give it a probability of zero.

And that's all there is to it.


## Training Code

Here is the code for training a language model, which implements *P(c* | *h)* with a counter of the number of times we have seen each character, for each history. The function `train_char_lm` takes a filename  to read the characters from. `order` is the history size to consult. Note that we pad the data with `order` leading characters so that we also learn how to start.


In [1]:
import random
import collections

In [2]:
class LanguageModel(collections.defaultdict):
    """A mapping from `order` history characters to a Counter of {'c': count} pairs,
    e.g., for order=4, {'spea': Counter({'k': 9, 'r': 1})}."""
    def __init__(self, order): 
        self.order = order
        self.default_factory = collections.Counter 

def train_char_lm(fname, order=4) -> LanguageModel:
    """Train an `order`-gram character-level language model on all the text in `fname`."""
    lm = LanguageModel(order)
    data = (order * PAD) + open(fname).read()
    for i in range(order, len(data)):
        history, char = data[i - order:i], data[i]
        lm[history][char] += 1
    for counter in lm.values():
        counter.total = sum(counter.values()) # Cache total counts (for sample_character)
    return lm

PAD = '`' # Character to pad the beginning of a text

Let's train it on Andrej's Shakespeare text:

In [3]:
! [ -f shakespeare_input.txt ] || curl -O https://norvig.com/ngrams/shakespeare_input.txt
! wc shakespeare_input.txt

  167204  832301 4573338 shakespeare_input.txt


In [4]:
lm = train_char_lm("shakespeare_input.txt", order=4)

Ok. Now let's do some queries:

In [5]:
lm['ello']

Counter({'r': 35,
         'w': 480,
         'u': 22,
         ',': 16,
         ' ': 8,
         '.': 4,
         '?': 4,
         ':': 3,
         'n': 1,
         "'": 10,
         '!': 4})

In [6]:
lm['Firs']

Counter({'t': 864})

In [7]:
lm['rst ']

Counter({'C': 119,
         'f': 14,
         'i': 21,
         't': 67,
         'u': 2,
         'S': 203,
         'h': 24,
         's': 41,
         'R': 1,
         'b': 31,
         'c': 16,
         'O': 23,
         'w': 30,
         'a': 28,
         'm': 28,
         'n': 25,
         'I': 12,
         'L': 133,
         'M': 74,
         'l': 13,
         'o': 38,
         'H': 5,
         'd': 19,
         'W': 42,
         'K': 10,
         'q': 2,
         'G': 112,
         'g': 14,
         'k': 5,
         'e': 4,
         'y': 3,
         'r': 9,
         'p': 11,
         'A': 7,
         'P': 18,
         'F': 15,
         'v': 3,
         'T': 4,
         'D': 4,
         'B': 12,
         'N': 1,
         "'": 1,
         'E': 2})

So `"ello"` is followed by either space, punctuation or `w` (or `r`, `u`, `n`), `"Firs"` is pretty much deterministic, and the word following `"rst "` can start with pretty much every letter.

## Generating from the model

To generate a random text from a model, we maintain a history of *order* characters, which starts with all pad characters. We then enter a loop that randomly samples a character from the history's counter, then updates the history by dropping its first character and adding the randomly-sampled character to the end. 

In [8]:
def generate_text(lm, length=1000) -> str:
    """Sample a random `length`-long passage from `lm`."""
    history = lm.order * PAD
    text = []
    for _ in range(length):
        c = sample_character(lm[history])
        history = history[1:] + c
        text.append(c)
    return ''.join(text)

To sample a single character from a counter, randomly choose an integer *n* from 1 to the total count of characters in the counter, then iterate through (character, count) items until the cumulative total of the counts meets or exceeds *n*.

In [9]:
def sample_character(counter) -> str:
    """Randomly sample the nth character from the counter."""
    n = random.randint(1, counter.total)
    cumulative = 0
    for ch in counter:
        cumulative += counter[ch]
        if cumulative >= n: 
            return ch

## Generated Shakespeare from different order models

Let's try to generate text based on different language-model orders. Let's start with something silly:

## order 2:

In [10]:
lm = train_char_lm("shakespeare_input.txt", order=2)
print(generate_text(lm))

Fireve notenink by in and my hillan foll harseceare.

Secers faing tom th, dre met speat's ano weauld derve,
OPHOPANG Roy, so grive us thishat me hall
That liall hen a
goin souren;' ther hou hust peavio.
BET:
Forat th ofend now bainch sed cur hou mat fatied shour men of
And ne hinese so may; somminto scall for taking iftlee the been siome this and naby thure dont.
RO:
Ther to-ny no ithe of I me drit, car! wer talk army deris to not ity
shearmonerstlet he gove proultre dink agiver cand swe ithe a glus,
SIDUKENE:
Yor.

Marrows anters, stemusere, bourects smennown here me thave ap asleand my forter tel
So liket wit.

Runt?

Nor me for pose
Def ing thempood I coame thoppestimp.
Tell ittle willy
As con
To thy soner
bless'd hasarcults what of thers nour
Why laillord!

MIL:
Forent itill down, ficke ord younsucks, lie dook:
Hear, a dill sty 't of togue; the I daus swast con now's anch lown re.
MENRY VIRANTO:
The ty, gonfectis me,
Tak Ridst
But all the of thothienday wilead him.

By to yess war

## order 4 

Order 2 was not so great... but what if we increase the order to 4?



In [11]:
lm = train_char_lm("shakespeare_input.txt", order=4)
print(generate_text(lm))

First Servantagem mayst can shall tends, the been of whether that senses of thou not her, sure that sure remain, thronet an expense.

Fool, her?
'Twas be told me?

SHALLOW:
De siege.

VALENTIO:
Never thy Montentend of rever my good quited, with laid best be? Tellingerous beseech were month.

KING CLAUDIUS:
He is faces.

CLAUDIUS:
In sure, and I hit foolish
As well teach yours, Here,
How does into thing more upon moves mother funerald it my very thank
That here often'd me.
Her hath broth
What wingeneral rob: if them here is jestion the baby you and thee.

HELENA:
O, their is,
And fig orders of fixed in these edges I would up, you this, for the good Service wilt seems to Caesar sing, smoking; and die, you may come wish these bed.

PETRUCHIO:
Falstaff ore to fast would I have show naked not what let us wrathe, and promio, thouse: and Mortime:
Both about of his that's hence how dukes inder you.

TRANIO:
At Melun.

Fourtier and take my some frust and my known
Ever her too.

TALBOT:
La man h



## order 7

Order 4 is already quite reasonable, and reads like English. Just 4 letters history! What if we increase it to 7?

In [12]:
lm = train_char_lm("shakespeare_input.txt", order=7)
print(generate_text(lm))

First Citizen:
Soft! take my castle call a new-devised impeach young one,
Were not
worth ten time to speak out,
Which Salique land:
Your mark his charge and sack great earnest of his practise, Gloucester, thou mean, he's hunted with this here stands,
'Tis one with such a question: his disturbed spirit, set a-work;
And so should I have in the liveries, all encertain money.

PERICLES:
How can these our favours
Have sat too curious, three thou wert as wise and right royal 'twas mine;
It is some taste of hers,
Hath seen such a place the event.
'Tis politic; he crosses to it, boy.

QUEEN MARGARET:
Give him,
for my battle's lost:
That well;
But bear my life.
I beg thy particular,--you had known well I shaked,
Which I feel.
I fight a question. Hubert, keep aloof at bay:
Sell ever soft hours,
Unless than thine own gladness he passage 'tis!--whose eyes can volley.

HAMLET:
How angerly.

PETRUCHIO:
Why, then it is
There were in our loves you,
A sea of glory, Gallia wars.

BASTARD:
Though he be n

## How about order 10?

In [13]:
lm = train_char_lm("shakespeare_input.txt", order=10)
print(generate_text(lm))

First Citizen:
Doth this news; yet what I have been a breakfast, washes his
hands, and save yourself!
My master, my dear Moth?

MOTH:
No, nor a man cannot make
him eat it that spake
that word?

QUINTUS:
Not so, an't please your highness, give us any thing for my labour by his own attaint?
'Tis doubt he will utter?

BRUTUS:
Hence! I will follow it! Come, and get to Naples? Keep in Tunis,
And Ferdinand, her brothers both from death,
But lusty, young, and of antiquity too; bawd-born.
Farewell, sweet Jack
Falstaff, where hath been prophesied France and Ireland
Bear that play;
For never yet a breaker of
proverbs: he will answer it. Some pigeons, Davy, a couple of Ford's
knaves, his hinds, bars me the poor
duke's office should do the duke of this place and great
ones I dare not: Sir Pierce of Exton, who
lately came from valiant Oxford?
How far hence
In mine own away;
But you gave in charge,
Is now dishonour me.

SUFFOLK:
Who waits there?

RODERIGO:
I know not; except, in that as ever
knapped

## This works pretty well

With an order of 4, we already get quite reasonable results. Increasing the order to 7 (about a word and a half of history) or 10 (about two short words of history) already gets us quite passable Shakepearan text. I'd say it is on par with the examples in Andrej's post. And how simple and un-mystical the model is!

## Aside: First words

One thing you may have noticed: all the generated passages start with "Fi". Why is that? Because the training data starts with the word "First" (preceeded by padding), and so when we go to randomly `generate_text`, the only thing that follows the initial padding is the word "First". We could get more variety in the generated text by breaking the training text up into sections  and inserting padding at the start of each section. But that would require some knowledge of the structure of the training text; right know the only assumption is that it is a sequence of characters.

In [14]:
! head shakespeare_input.txt

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:


## So why am I impressed with the RNNs after all?

Generating English a character at a time -- not so impressive in my view. The RNN needs to learn the previous *n* letters, for a rather small *n*, and that's it. 

However, Karpathy's C++ code generation example is very impressive. Why? because of the context awareness. Note that in all of Karpathy's posted examples, the code is well indented, the braces and brackets are correctly nested, and even the comments start and end correctly. This is not something that can be achieved by simply looking at the previous *n* characters. 

If Karpathy's examples are not cherry-picked, and the output is generally that nice, then the LSTM did learn something not trivial at all.

# Linux Kernel C++ Code

Just for the fun of it, let's see what our simple language model does with the Linux-kernel code:

In [15]:
! [ -f linux_input.txt ] || curl -O https://norvig.com/ngrams/linux_input.txt
! wc linux_input.txt

  241465  759639 6206997 linux_input.txt


In [16]:
lm = train_char_lm("linux_input.txt", order=10)
print(generate_text(lm))

/*
 * linux/kernel_stat.h>

int rcu_cpu_notify(CPU_CLUSTER_PM_EXIT, -1, NULL);
}

bool __weak is_swbp_insn
 * Returns:
 *	Zero for success, 0 (invalid alignment with below mechanisms in the kprobe gone and remove
 * @action: action to SIG_DFL for a signal frame,
	   and will be woken
 * up from TASK_TRACED);
	if (mod->state == MODULE_STATE_GOING:
		state = possible;
			break;
			rest++;
		}
		/*
		 * Another cpu said 'go' */
		/* Still using kdb, this problematic.
 */
void init_sched_dl_class(void)
{
	field_cachep = KMEM_CACHE(vm_area_struct *vma, struct autogroup *autogroup_kref_put(ag);

	return 0;
}

void do_raw_write_can_lock(l)	write_can_lock(l)

/*
 * Software Foundation, and allows traversing */
static int perf_output_sample_rate __read_mostly futex_hash_bucket(futex));
 *     smp_mb() anyway for documentation of its parent
	 * using getname.
 *
 * default: 5 msec, units: microseconds, all such timers will fire
 * at the end of this function implementation
 * @num: number of tot

In [17]:
lm = train_char_lm("linux_input.txt", order=15)
print(generate_text(lm))

/*
 * linux/kernel/irq/handle.c
 *
 * Copyright (C) 2012 Dario Faggioli <raistlin@linux.it>,
 *                  |         |\n");
}

static int __read_mostly =
{
	.name		= "irqsoff",
	.init		= preemptoff_tracer __read_mostly sysctl_hung_task_timeout_secs(struct ctl_table *table,
		  int write, void *data),
		 struct lock_list *unsafe_entry,
			struct lock_class_key irq_desc_lock_class;

#if defined(CONFIG_RCU_TRACE */

static __init int user_namespace *ns = task_active_pid_ns(parent));
	info.si_uid = from_kuid_munged(current_user_ns(), task_uid(p));
	struct siginfo info;

	if (argc != 1)
		return -EINVAL;
	}
#endif

#ifdef CONFIG_SMP

static void
irq_thread_check_affinity(struct rcu_node *rnp)
{
	return rnp->gp_tasks != NULL;
}

/*
 * return non-zero if there is a SIGKILL that should be deferred, ETT_NONE if nothing to defer.
 */
enum event_trigger_free() (see
 *	trace_event_trigger_enable_disable(file, 0);
			break;
		}

		/*
		 * Wait for all preempt-disabled section so we can as wel

In [18]:
lm = train_char_lm("linux_input.txt", order=20)
print(generate_text(lm))

/*
 * linux/kernel/itimer.c
 *
 * Copyright (C) 2008 Red Hat, Inc., Peter Zijlstra <pzijlstr@redhat.com>
 *  Copyright (C) 2004, 2005, 2006 Red Hat, Inc., Ingo Molnar <mingo@redhat.com>
 *  Copyright (C) 2004 IBM Corporation
 *
 *  Author: Serge Hallyn <serue@us.ibm.com>
 *
 *  This program is distributed in the hope that it will be useful,
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 * GNU General Public License as
 *  published by the Free Software Foundation; version 2
 * of the License, or (at
 * your option) any later version.
 *
 * This program is free software; you can redistribute it and/or
 * modify it under the terms of the GNU GPL, version 2
 *
 * This file implements counting semaphores.
 * A counting semaphore may be acquired 'n' times before sleeping.
 * See mutex.c for single-acquisition sleeping locks which enforce
 * rules which allow code to be debugged more easily.
 */

/*
 * Some 

In [19]:
print(generate_text(lm))

/*
 * linux/kernel/irq/manage.c
 *
 * Copyright (C) 2008-2014 Mathieu Desnoyers
 *
 * This program is distributed in the hope that it will be useful, but
 * WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
 * FITNESS FOR A PARTICULAR PURPOSE.  See the
 * GNU General Public Licence
 * as published by the Free Software Foundation, version 2 of the
 *  License.
 *
 *  Jun 2006 - namespaces support
 *             OpenVZ, SWsoft Inc.
 *    (C) 2007 Sukadev Bhattiprolu <sukadev@us.ibm.com>, IBM
 *     Many thanks to Oleg Nesterov for comments and help
 *
 */

#include <linux/irq_work.h>
#include <linux/syscalls.h>
#include <linux/swapops.h>
#include <linux/slab.h>
#include <linux/module.h>
#include <linux/kmod.h>
#include <linux/sched.h>	/* for cond_resched */
#include <linux/types.h>
#include <linux/mutex.h>
#include <linux/proc_fs.h>
#include <linux/memory.h>
#include <linux/task_io_accounting_ops.h>
#include <linux/proc_fs.h>
#include <linux/highmem.h>
#includ

In [20]:
print(generate_text(lm, length=5000))

/*
 * linux/kernel/irq/manage.c
 *
 * Copyright (C) 2010 Red Hat, Inc.
 *
 * Note: Most of this code is borrowed heavily from the original softlockup
 * detector, so thanks to Ingo for the initial implementation (includes suggestions from
 *		Rusty Russell).
 * 2004-Aug	Updated by Prasanna S Panchamukhi <prasanna@in.ibm.com> Changed Kprobes
 *		exceptions notifier as suggested by Andi Kleen.
 * 2004-July	Suparna Bhattacharya <suparna@in.ibm.com> added jumper probes
 *		interface to access function arguments.
 * 2004-Sep	Prasanna S Panchamukhi
 *		<prasanna@in.ibm.com> with
 *		hlists and exceptions notifier to be first on the priority list.
 * 2005-May	Hien Nguyen <hien@us.ibm.com>, Jim Keniston
 *		<jkenisto@us.ibm.com> and Prasanna S Panchamukhi <prasanna@in.ibm.com> Changed Kprobes
 *		exceptions notifier as suggested by Andi Kleen.
 * 2004-July	Suparna Bhattacharya <suparna@in.ibm.com> added function-return probe instances associated with the event
 *
 * If an event has triggers an

## Analysis

Order 10 is pretty much junk. In order 15 things sort-of make sense, but we jump abruptly between the `[sic]`
and by order 20 we are doing quite nicely — but are far from keeping good indentation and brackets. 

How could we? we do not have the memory, and these things are not modeled at all. While we could quite easily enrich our model to support also keeping track of brackets and indentation (by adding information such as "have I seen ( but not )" to the conditioning history), this requires extra work, non-trivial human reasoning, and will make the model significantly more complex. 

Karpathy's LSTM, on the other hand, seemed to have just learn it on its own. And that's impressive.